In [ ]:
%pip install fastbook transformers tokenizers==0.12.0 torch

In [14]:

import fastbook
import torch
from fastai.vision.all import *
from fastbook import *
import numpy as np
import numpy.random as nprnd

fastbook.setup_book()
model_name = "microsoft/deberta-v3-small"

In [29]:
from datasets import load_dataset,Dataset
from transformers import AutoTokenizer
import pandas as pd
tokenizer = AutoTokenizer.from_pretrained(model_name)
from datasets import load_dataset

def concat_columns(example):
    example['input'] = 'TEXT1: ' + example["context"] + '; TEXT2: ' + example["target"] + '; ANC1: ' + example["anchor"]
    return example

def process_dataset(file):
    df = pd.read_csv(file)
    df["input"] = 'TEXT1: ' + df["context"] + '; TEXT2: ' + df["target"] + '; ANC1: ' + df["anchor"]
    train_dataset = Dataset.from_pandas(df)
    return train_dataset
test_dataset = process_dataset('./test.csv')
train_dataset = process_dataset('./train.csv')


loading configuration file https://huggingface.co/microsoft/deberta-v3-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/8e0c12a7672d1d36f647c86e5fc3a911f189d8704e2bc94dde4a1ffe38f648fa.9df96bac06c2c492bc77ad040068f903c93beec14607428f25bf9081644ad0da
Model config DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-small",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att

In [30]:
def encode(examples):
    return tokenizer(examples['input'])

train_dataset = train_dataset.map(encode, batched=True)
test_dataset = test_dataset.map(encode, batched=True)
print(test_dataset.column_names)

  0%|          | 0/37 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

['id', 'anchor', 'target', 'context', 'input', 'input_ids', 'token_type_ids', 'attention_mask']


In [31]:
train_dataset = train_dataset.rename_columns({'score':'labels'})


ValueError: Original column names {'score'} not in the dataset. Current columns in the dataset: ['id', 'anchor', 'target', 'context', 'input', 'input_ids', 'token_type_ids', 'attention_mask']

In [ ]:
def corr(x,y): return np.corrcoef(x,y)[0][1]
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

In [32]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
lr = 8e-5
bs = 128
epochs = 4
# Define the training arguments
training_args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

# Create the Trainer and train the model
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,          # evaluation dataset,
    tokenizer=tokenizer, 
    compute_metrics=corr_d
)

trainer.train()



loading configuration file https://huggingface.co/microsoft/deberta-v3-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/8e0c12a7672d1d36f647c86e5fc3a911f189d8704e2bc94dde4a1ffe38f648fa.9df96bac06c2c492bc77ad040068f903c93beec14607428f25bf9081644ad0da
Model config DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-small",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input"

Step,Training Loss
500,0.068400
1000,0.036800
1500,0.030800
2000,0.028600
2500,0.023900
3000,0.020400
3500,0.020000
4000,0.018800
4500,0.017900
5000,0.013800


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1500/special_toke

TrainOutput(global_step=6840, training_loss=0.023783158070859855, metrics={'train_runtime': 741.353, 'train_samples_per_second': 147.594, 'train_steps_per_second': 9.226, 'total_flos': 640294888782390.0, 'train_loss': 0.023783158070859855, 'epoch': 3.0})

In [36]:
preds = trainer.predict(test_dataset).predictions.astype(float)
preds

The following columns in the test set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: target, anchor, context, input, id. If target, anchor, context, input, id are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 36
  Batch size = 64


AttributeError: 'numpy.ndarray' object has no attribute 'precision'

In [35]:
preds = np.clip(preds, 0, 1)
preds

array([[0.54668516],
       [0.67762357],
       [0.63412821],
       [0.29955265],
       [0.        ],
       [0.49668178],
       [0.51377422],
       [0.        ],
       [0.29256913],
       [1.        ],
       [0.28536856],
       [0.24432719],
       [0.77080995],
       [0.88923609],
       [0.77721518],
       [0.34552768],
       [0.32667062],
       [0.        ],
       [0.53141445],
       [0.31685403],
       [0.4005917 ],
       [0.27164331],
       [0.02667243],
       [0.23149383],
       [0.55498201],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.6789816 ],
       [0.25649375],
       [0.        ],
       [0.65520978],
       [0.5595926 ],
       [0.35984644],
       [0.2465352 ]])